In [6]:
!pip install split-folders

In [7]:
import os
import glob
import splitfolders
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

from tensorflow.keras.utils import to_categorical

/home/daabv/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2022-10-09 19:31:09.074363: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-09 19:31:09.892406: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-09 19:31:09.892440: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-0

In [9]:
input_train_path  = "/media/daabv/My Passport/mri_data/input/training/"
input_test_path   = "/media/daabv/My Passport/mri_data/input/validation/"

In [24]:
def train_nii2npy(path):
    
    t2_list    = sorted(glob.glob(path + "/*/*t2.nii"))
    t1ce_list  = sorted(glob.glob(path + "/*/*t1ce.nii"))
    flair_list = sorted(glob.glob(path + "/*/*flair.nii"))
    mask_list  = sorted(glob.glob(path + "/*/*seg.nii"))
    
    for i, img in enumerate(flair_list):

        temp_img_t2 = nib.load(t2_list[i]).get_fdata()
        temp_img_t2 = scaler.fit_transform(
            temp_img_t2.reshape(-1, temp_img_t2.shape[-1])).reshape(temp_img_t2.shape)

        temp_img_t1ce = nib.load(t1ce_list[i]).get_fdata()
        temp_img_t1ce = scaler.fit_transform(
            temp_img_t1ce.reshape(-1, temp_img_t1ce.shape[-1])).reshape(temp_img_t1ce.shape)

        temp_img_flair = nib.load(flair_list[i]).get_fdata()
        temp_img_flair = scaler.fit_transform(
            temp_img_flair.reshape(-1, temp_img_flair.shape[-1])).reshape(temp_img_flair.shape)
        
        temp_mask = nib.load(mask_list[i]).get_fdata()
        temp_mask = temp_mask.astype(np.uint8)
        temp_mask[temp_mask == 4] = 3

        temp_combined_images = np.stack(
            [temp_img_flair, temp_img_t1ce, temp_img_t2], axis=3)

        temp_combined_images = temp_combined_images[56:184, 56:184, 13:141]
        temp_mask = temp_mask[56:184, 56:184, 13:141]

        val, counts = np.unique(temp_mask, return_counts=True)

        if (1 - (counts[0]/counts.sum())) > 0.01:
            temp_mask = to_categorical(temp_mask, num_classes=4)
            np.save('/media/daabv/My Passport/mri_data/output/training/images/image_' + str(i) + '.npy', temp_combined_images)
            np.save('/media/daabv/My Passport/mri_data/output/training/masks/mask_' + str(i) + '.npy', temp_mask)

        else:
            pass

In [28]:
def test_nii2npy(path):
    
    t2_list    = sorted(glob.glob(path + "/*/*t2.nii"))
    t1ce_list  = sorted(glob.glob(path + "/*/*t1ce.nii"))
    flair_list = sorted(glob.glob(path + "/*/*flair.nii"))
    mask_list  = sorted(glob.glob(path + "/*/*seg.nii"))
    
    for i, img in enumerate(flair_list):

        temp_img_t2 = nib.load(t2_list[i]).get_fdata()
        temp_img_t2 = scaler.fit_transform(
            temp_img_t2.reshape(-1, temp_img_t2.shape[-1])).reshape(temp_img_t2.shape)

        temp_img_t1ce = nib.load(t1ce_list[i]).get_fdata()
        temp_img_t1ce = scaler.fit_transform(
            temp_img_t1ce.reshape(-1, temp_img_t1ce.shape[-1])).reshape(temp_img_t1ce.shape)

        temp_img_flair = nib.load(flair_list[i]).get_fdata()
        temp_img_flair = scaler.fit_transform(
            temp_img_flair.reshape(-1, temp_img_flair.shape[-1])).reshape(temp_img_flair.shape)
        
        temp_combined_images = np.stack(
            [temp_img_flair, temp_img_t1ce, temp_img_t2], axis=3)

        temp_combined_images = temp_combined_images[56:184, 56:184, 13:141]

        np.save('/media/daabv/My Passport/mri_data/output/test/images/image_' + str(i) + '.npy', temp_combined_images)

In [30]:
train_nii2npy(input_train_path)
valid_nii2npy(input_val_path)

In [13]:
input_folder = "/media/daabv/My Passport/mri_data/output/training/"
output_folder = "/media/daabv/My Passport/mri_data/output/validation/"

splitfolders.ratio(input_folder, output=output_folder, ratio=(.75, .25), group_prefix=None)



Copying files: 0 files [00:00, ? files/s]

Copying files: 2 files [00:00,  2.77 files/s]

Copying files: 3 files [00:01,  2.14 files/s]

Copying files: 4 files [00:03,  1.17s/ files]

Copying files: 5 files [00:04,  1.01 files/s]

Copying files: 6 files [00:05,  1.09 files/s]

Copying files: 7 files [00:07,  1.51s/ files]

Copying files: 8 files [00:08,  1.27s/ files]

Copying files: 9 files [00:09,  1.12s/ files]

Copying files: 10 files [00:12,  1.67s/ files]

Copying files: 11 files [00:13,  1.38s/ files]

Copying files: 12 files [00:14,  1.35s/ files]

Copying files: 13 files [00:15,  1.30s/ files]

Copying files: 14 files [00:17,  1.41s/ files]

Copying files: 15 files [00:20,  1.89s/ files]

Copying files: 16 files [00:21,  1.68s/ files]

Copying files: 17 files [00:22,  1.50s/ files]

Copying files: 18 files [00:23,  1.29s/ files]

Copying files: 19 files [00:24,  1.34s/ files]

Copying files: 20 files [00:28,  1.94s/ files]

Copying files: 21 files [00:28,  1.56s/ files]

Cop

Copying files: 151 files [03:49,  1.68s/ files]

Copying files: 152 files [03:50,  1.38s/ files]

Copying files: 153 files [03:51,  1.30s/ files]

Copying files: 154 files [03:53,  1.38s/ files]

Copying files: 155 files [03:56,  1.83s/ files]

Copying files: 156 files [03:58,  1.88s/ files]

Copying files: 157 files [03:58,  1.51s/ files]

Copying files: 158 files [03:59,  1.25s/ files]

Copying files: 159 files [04:01,  1.59s/ files]

Copying files: 160 files [04:04,  2.03s/ files]

Copying files: 161 files [04:05,  1.63s/ files]

Copying files: 162 files [04:06,  1.33s/ files]

Copying files: 163 files [04:07,  1.27s/ files]

Copying files: 164 files [04:08,  1.33s/ files]

Copying files: 165 files [04:10,  1.39s/ files]

Copying files: 166 files [04:11,  1.43s/ files]

Copying files: 167 files [04:15,  2.17s/ files]

Copying files: 168 files [04:16,  1.75s/ files]

Copying files: 169 files [04:17,  1.43s/ files]

Copying files: 170 files [04:18,  1.31s/ files]

Copying files: 171 f

Copying files: 299 files [07:42,  1.43s/ files]

Copying files: 300 files [07:43,  1.33s/ files]

Copying files: 301 files [07:47,  1.86s/ files]

Copying files: 302 files [07:47,  1.51s/ files]

Copying files: 303 files [07:48,  1.26s/ files]

Copying files: 304 files [07:51,  1.83s/ files]

Copying files: 305 files [07:52,  1.49s/ files]

Copying files: 306 files [07:52,  1.25s/ files]

Copying files: 307 files [07:54,  1.20s/ files]

Copying files: 308 files [07:55,  1.27s/ files]

Copying files: 309 files [07:57,  1.34s/ files]

Copying files: 310 files [07:58,  1.38s/ files]

Copying files: 311 files [08:02,  2.09s/ files]

Copying files: 312 files [08:03,  1.70s/ files]

Copying files: 313 files [08:03,  1.38s/ files]

Copying files: 314 files [08:06,  1.76s/ files]

Copying files: 315 files [08:07,  1.68s/ files]

Copying files: 316 files [08:08,  1.40s/ files]

Copying files: 317 files [08:09,  1.20s/ files]

Copying files: 318 files [08:12,  1.83s/ files]

Copying files: 319 f

Copying files: 447 files [10:54,  1.04s/ files]

Copying files: 448 files [10:57,  1.54s/ files]

Copying files: 449 files [10:58,  1.29s/ files]

Copying files: 450 files [10:59,  1.12s/ files]

Copying files: 451 files [10:59,  1.00 files/s]

Copying files: 452 files [11:00,  1.01 files/s]

Copying files: 453 files [11:03,  1.52s/ files]

Copying files: 454 files [11:04,  1.29s/ files]

Copying files: 455 files [11:04,  1.12s/ files]

Copying files: 456 files [11:05,  1.02 files/s]

Copying files: 457 files [11:06,  1.21 files/s]

Copying files: 458 files [11:08,  1.46s/ files]

Copying files: 459 files [11:09,  1.25s/ files]

Copying files: 460 files [11:10,  1.08s/ files]

Copying files: 461 files [11:11,  1.05 files/s]

Copying files: 462 files [11:13,  1.39s/ files]

Copying files: 463 files [11:14,  1.19s/ files]

Copying files: 464 files [11:14,  1.05s/ files]

Copying files: 465 files [11:15,  1.08 files/s]

Copying files: 466 files [11:16,  1.18 files/s]

Copying files: 467 f

Copying files: 595 files [13:55,  1.46s/ files]

Copying files: 596 files [13:56,  1.27s/ files]

Copying files: 597 files [13:57,  1.09s/ files]

Copying files: 598 files [13:58,  1.05 files/s]

Copying files: 599 files [13:58,  1.15 files/s]

Copying files: 600 files [13:59,  1.10 files/s]

Copying files: 601 files [14:00,  1.03 files/s]

Copying files: 602 files [14:03,  1.42s/ files]

Copying files: 603 files [14:05,  1.51s/ files]

Copying files: 604 files [14:05,  1.24s/ files]

Copying files: 605 files [14:06,  1.05s/ files]

Copying files: 606 files [14:06,  1.09 files/s]

Copying files: 607 files [14:07,  1.19 files/s]

Copying files: 608 files [14:08,  1.11 files/s]

Copying files: 609 files [14:09,  1.04 files/s]

Copying files: 610 files [14:10,  1.01 files/s]

Copying files: 611 files [14:11,  1.00 files/s]

Copying files: 612 files [14:12,  1.00s/ files]

Copying files: 613 files [14:14,  1.28s/ files]

Copying files: 614 files [14:17,  1.67s/ files]

Copying files: 615 f